# 순환 신경망을 사용해 다음 단어를 예측하는 언어 모델링

In [1]:
from __future__ import print_function
import os

from sklearn.model_selection import train_test_split
import nltk
import numpy as np
import string

# 파일 읽기
with open('alice_in_wonderland.txt', 'r') as content_file:
    content = content_file.read()
content2 = " ".join("".join([" " if ch in string.punctuation else ch for ch in content]).split())

tokens = nltk.word_tokenize(content2)
tokens = [word.lower() for word in tokens if len(word)>=2]

# N-1이 마지막 N번째 단어를 예측하는 데 사용되는 N그램에 대한 N값 선택
N = 3
quads = list(nltk.ngrams(tokens,N))
newl_app = []
for ln in quads:
    newl = " ".join(ln)
    newl_app.append(newl)

In [2]:
# 단어 벡터화
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()

x_trigm = []
y_trigm = []

for l in newl_app:
    x_str = " ".join(l.split()[0:N-1])
    y_str = l.split()[N-1]
    x_trigm.append(x_str)
    y_trigm.append(y_str)
    
x_trigm_check = vectorizer.fit_transform(x_trigm).todense()
y_trigm_check = vectorizer.fit_transform(y_trigm).todense()

In [4]:
# 단어에서 정수, 정수에서 단어로 변환하기 위한 딕셔너리
dictnry = vectorizer.vocabulary_
rev_dictnry = {v:k for k,v in dictnry.items()}

X = np.array(x_trigm_check)
Y = np.array(y_trigm_check)

Xtrain, Xtest, Ytrain, Ytest,xtrain_tg,xtest_tg = train_test_split(X, Y,x_trigm, test_size=0.3,random_state=42)

print("X Train shape",Xtrain.shape, "Y Train shape" , Ytrain.shape)
print("X Test shape",Xtest.shape, "Y Test shape" , Ytest.shape)

X Train shape (17947, 2559) Y Train shape (17947, 2559)
X Test shape (7692, 2559) Y Test shape (7692, 2559)


In [5]:
# 모델 구축
from keras.layers import Input,Dense,Dropout
from keras.models import Model
np.random.seed(42)
BATCH_SIZE = 128
NUM_EPOCHS = 100

input_layer = Input(shape = (Xtrain.shape[1],),name="input")
first_layer = Dense(1000,activation='relu',name = "first")(input_layer)
first_dropout = Dropout(0.5,name="firstdout")(first_layer)

second_layer = Dense(800,activation='relu',name="second")(first_dropout)

third_layer = Dense(1000,activation='relu',name="third")(second_layer)
third_dropout = Dropout(0.5,name="thirdout")(third_layer)

fourth_layer = Dense(Ytrain.shape[1],activation='softmax',name = "fourth")(third_dropout)
history = Model(input_layer,fourth_layer)
history.compile(optimizer = "adam",loss="categorical_crossentropy",metrics=["accuracy"])

print (history.summary())

Using TensorFlow backend.


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 2559)              0         
_________________________________________________________________
first (Dense)                (None, 1000)              2560000   
_________________________________________________________________
firstdout (Dropout)          (None, 1000)              0         
_________________________________________________________________
second (Dense)               (None, 800)               800800    
_________________________________________________________________
third (Dense)                (None, 1000)              801000    
_________________________________________________________________
thirdout (Dropout)           (None, 1000)              0         
_________________________________________________________________
fourth (Dense)               (None, 2559)              2561

In [6]:
# 모델 학습
history.fit(Xtrain, Ytrain, batch_size=BATCH_SIZE,epochs=NUM_EPOCHS, verbose=1,validation_split = 0.2)

Train on 14357 samples, validate on 3590 samples
Epoch 1/100
14357/14357 [==============================] - 18s 1ms/step - loss: 6.3383 - accuracy: 0.0547 - val_loss: 6.0992 - val_accuracy: 0.0696
Epoch 2/100
14357/14357 [==============================] - 16s 1ms/step - loss: 5.8994 - accuracy: 0.0678 - val_loss: 6.0303 - val_accuracy: 0.0749
Epoch 3/100
14357/14357 [==============================] - 17s 1ms/step - loss: 5.6784 - accuracy: 0.0849 - val_loss: 5.9898 - val_accuracy: 0.0852
Epoch 4/100
14357/14357 [==============================] - 16s 1ms/step - loss: 5.4228 - accuracy: 0.1089 - val_loss: 6.0003 - val_accuracy: 0.0950
Epoch 5/100
14357/14357 [==============================] - 16s 1ms/step - loss: 5.1663 - accuracy: 0.1319 - val_loss: 5.9989 - val_accuracy: 0.1014
Epoch 6/100
14357/14357 [==============================] - 14s 998us/step - loss: 4.9276 - accuracy: 0.1523 - val_loss: 6.0323 - val_accuracy: 0.1064
Epoch 7/100
14357/14357 [==============================] - 17

14357/14357 [==============================] - 11s 734us/step - loss: 1.3331 - accuracy: 0.6095 - val_loss: 10.9615 - val_accuracy: 0.1056
Epoch 56/100
14357/14357 [==============================] - 11s 779us/step - loss: 1.3319 - accuracy: 0.6093 - val_loss: 11.0427 - val_accuracy: 0.1019
Epoch 57/100
14357/14357 [==============================] - 10s 725us/step - loss: 1.3295 - accuracy: 0.6065 - val_loss: 10.9671 - val_accuracy: 0.1022
Epoch 58/100
14357/14357 [==============================] - 11s 733us/step - loss: 1.3147 - accuracy: 0.6144 - val_loss: 10.8870 - val_accuracy: 0.1022
Epoch 59/100
14357/14357 [==============================] - 11s 746us/step - loss: 1.3075 - accuracy: 0.6141 - val_loss: 10.9896 - val_accuracy: 0.1058
Epoch 60/100
14357/14357 [==============================] - 11s 760us/step - loss: 1.2890 - accuracy: 0.6166 - val_loss: 11.0631 - val_accuracy: 0.1017
Epoch 61/100
14357/14357 [==============================] - 10s 712us/step - loss: 1.2918 - accuracy:

In [8]:
# 모델 예측
Y_pred = history.predict(Xtest)

# 테스트 데이터에 대한 샘플 확인
print ("Prior bigram words","|Actual","|Predicted","\n")
for i in range(10):
    print (i,xtest_tg[i],"|",rev_dictnry[np.argmax(Ytest[i])],"|",rev_dictnry[np.argmax(Y_pred[i])])

Prior bigram words |Actual |Predicted 

0 the evening | beautiful | beautiful
1 slipped in | like | the
2 alice swallowing | down | her
3 an encouraging | tone | mouse
4 waistcoat pocket | or | grunted
5 she went | on | on
6 that she | knew | was
7 down on | her | one
8 dormouse went | on | on
9 soup soup | of | and


In [9]:
import random
NUM_DISPLAY = 10
for i in random.sample(range(len(xtest_tg)),NUM_DISPLAY):
    print (i,xtest_tg[i],"|",rev_dictnry[np.argmax(Ytest[i])],"|",rev_dictnry[np.argmax(Y_pred[i])])

4221 pieces against | one | of
5099 english now | opening | thought
4254 the jury | who | box
2722 with alice | waited | love
2086 fly and | the | feet
1465 why with | an | there
1137 among the | trees | party
1435 at alice | for | not
487 make you | dry | present
5389 box with | the | one
